In [1]:
from src.schemas.thesaurus.mads import SchemaMads
from datetime import datetime
from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [2]:
req = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Authority",
                "label": "Authority"
            }
        },
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            }
        }
    ],
    "isMemberOfMADSCollection": [
        {
            "collection": {
                "value": "names",
                "label": ""
            }
        }
    ],
    "elementList": [
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                "label": "FullNameElement"
            },
            "elementValue": "Borges, Jorge Luis,",
            "elementLang": {
                "value": "",
                "label": ""
            }
        },
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                "label": "DateNameElement"
            },
            "elementValue": "1899-1986",
            "elementLang": {
                "value": "",
                "label": ""
            }
        }
    ],
    "authoritativeLabel": {
        "value": "Borges, Jorge Luis, 1899-1986"
    },
    "birth": {
        "place": "Buenos Aires (Argentina)",
        "day": "24",
        "month": {
            "value": "08",
            "label": "Agosto"
        },
        "year": "1899"
    },
    "death": {
        "day": "26",
        "month": {
            "value": "03",
            "label": "Março"
        },
        "year": "1986"
    },
    "hasVariant": [
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Borges, J. L.",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "(Jorge Luis),",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Borhes, Horhe Luis,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Borkhes, Khorkhe Luyis,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Borches, Chorche Louis,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Борхес, Хорхе Луис,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "בארכעס, כארכע לויס",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "בארכעס, כארכע לויס,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899־",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "בורחס, חורחה לואיס",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "博尔赫斯,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#TermsOfAddressNameElement",
                        "label": "TermsOfAddressNameElement"
                    },
                    "elementValue": "J. L,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Borges Acevedo, Jorge Luis,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "ホルヘ・ルイス・ボルヘス,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Hao'erhe Luyisi Bo'erhesi,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "豪尔赫·路易斯·博尔赫斯,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "Būrkhīs, Khūrkhī Lūwis,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#PersonalName",
                "label": "PersonalName"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#FullNameElement",
                        "label": "FullNameElement"
                    },
                    "elementValue": "بورخيس, خورخي لويس,",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#DateNameElement",
                        "label": "DateNameElement"
                    },
                    "elementValue": "1899-1986",
                    "elementLang": {
                        "value": "",
                        "label": ""
                    }
                }
            ]
        }
    ],
    "hasCloseExternalAuthority": [
        {
            "base": "vocab.getty.edu",
            "label": "Borges, Jorge Luis",
            "uri": "http://vocab.getty.edu/ulan/500006476"
        },
        {
            "base": "d-nb.info",
            "label": "Borges, Jorge Luis",
            "uri": "https://d-nb.info/gnd/118513532"
        },
        {
            "base": "id.worldcat.org",
            "label": "Borges, Jorge Luis, 1899-1986",
            "uri": "http://id.worldcat.org/fast/30252"
        },
        {
            "base": "www.wikidata.org",
            "label": "Jorge Luis Borges",
            "uri": "http://www.wikidata.org/entity/Q909"
        }
    ],
    "identifiersLccn": "n79007035"
}

In [3]:
request = SchemaMads(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = "10"

In [14]:
if request.birth.day and request.birth.month.value and request.birth.year:
    date = f"{request.birth.day}-{request.birth.month.value}-{request.birth.year}"
    print(date)

24-08-1899


In [12]:
request.birth.month.value

'08'

In [17]:
def MakeDoc(request):
    # authority = request.authoritativeLabel.value
    # authority = authority.removesuffix(',')
    isMemberOfMADSCollection = [i.collection.value for i in request.isMemberOfMADSCollection]

    doc = { 
            'id': f'authority#{request.adminMetadata.identifiedBy}',
            'type': [i.type.label for i in request.resource],
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": request.authoritativeLabel,
            "authority": request.authoritativeLabel.value,
            "isMemberOfMADSCollection": isMemberOfMADSCollection
        }
    # Lang
    element = request.elementList[0]
    lang = element.elementLang.label
    if lang != "":
        doc['lang'] = lang

    if request.identifiersLccn:
        doc['identifiersLccn'] = request.identifiersLccn
        
    if request.imagem:
        doc['imagem'] = request.imagem

    if request.adminMetadata.changeDate:
        doc['changeDate'] = request.adminMetadata.changeDate.strftime("%Y-%m-%dT%H:%M:%S")
    
    if request.fullerName:
        doc['fullerName'] = request.fullerName
    
    metadados = ['birthDayDate', 'birthMonthDate','birthYearDate', 'birthDate', 'birthPlace', 'deathDate', 'deathPlace',
                 'deathDayDate', 'deathMonthDate', 'deathYearDate']
    for metadado in metadados:
        value = request.model_dump().get(metadado)
        if value:
            doc[metadado] = value

    if request.birth.day and request.birth.month.value and request.birth.year:
        date = f"{request.birth.day}-{request.birth.month.value}-{request.birth.year}"
        doc['birthDate'] = date
    elif request.birth.month.value and request.birth.year:
        date = f"{request.birth.month.value}-{request.birth.year}"
        doc['birthDate'] = date 
    elif request.request.birth.year:
        doc['birthDate'] = request.request.birth.year 

    if request.death.day and request.death.month.value and request.death.year:
        date = f"{request.death.day}-{request.death.month.value}-{request.death.year}"
        doc['deathhDate'] = date
    elif request.death.month.value and request.death.year:
        date = f"{request.death.month.value}-{request.death.year}"
        doc['deathDate'] = date 
    elif request.request.death.year:
        doc['deathDate'] = request.request.death.year 
    
    # hasAffiliation  
    if request.hasAffiliation:
        affiliations = list()
        for i in request.hasAffiliation:
            if i.organization.uri:
                a = {
                'id': f"authority#{request.identifiersLocal}/hasAffiliation#{i.organization.uri.split('/')[-1]}",
                'organization': {
                    "label": i.organization.label,
                    'uri': i.organization.uri,
                    "base": i.organization.base
                    },                
                'affiliationStart': i.affiliationStart }
            else:
                a = {
                'id': f"authority#{request.identifiersLocal}/hasAffiliation#{i.organization.label}",
                'organization': {
                    "label": i.organization.label,
                    # 'uri': i.organization.uri,
                    "base": i.organization.base
                    },                
                'affiliationStart': i.affiliationStart }

            if i.affiliationEnd:
                a['affiliationEnd'] = i.affiliationEnd
            affiliations.append(a)
        doc['hasAffiliation'] = affiliations
        doc['affiliation']  = [i['organization'] for i in affiliations]

    # hasVariant
    if request.hasVariant:
        variants = list()
        hasVariants = list()
        for i in request.hasVariant:
            print(i) 
            label = [j.elementValue.value for j in i.elementList]
            variantLabel = " ".join(label)
            variants.append(variantLabel)
            hasVariant = i.model_dump()
            hasVariant['variantLabel'] = variantLabel
            hasVariants.append(hasVariant)
        doc['variant'] = variants
        doc['hasVariant'] = hasVariants

    # hasCloseExternalAuthority
    if request.hasCloseExternalAuthority:
        uris = list()
        for i in request.hasCloseExternalAuthority:
            uri = {
                    'id': f"authority#{request.identifiersLocal}/hasCloseExternalAuthority#{i.uri.split('/')[-1]}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base }
            uris.append(uri)
        doc['hasCloseExternalAuthority'] = uris

    # Broader
    if request.hasBroaderAuthority:
        listMads = list()
        for i in request.hasBroaderAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasBroaderAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasBroaderAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasBroaderAuthority'] = listMads
        doc['hasBroaderAuthorityLabels']  = [i['label'] for i in listMads]

    # hasNarrowerAuthority
    if request.hasNarrowerAuthority:
        listMads = list()
        for i in request.hasNarrowerAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasNarrowerAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasNarrowerAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasNarrowerAuthority'] = listMads
        doc['hasNarrowerAuthorityLabels']  = [i['label'] for i in listMads]

    # hasReciprocalAuthority
    if request.hasReciprocalAuthority:
        listMads = list()
        for i in request.hasReciprocalAuthority:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasReciprocalAuthority#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/hasReciprocalAuthority#{i.label}",
                    'label': i.label,
                    'base': i.base }

            listMads.append(uri)
        doc['hasReciprocalAuthority'] = listMads
        doc['hasReciprocalAuthorityLabels']  = [i['label'] for i in listMads]
        
    # Occupation
    if request.occupation:
        occupations = list()
        for i in request.occupation:
            if i.uri:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/occupation#{i.uri.split('/')[-1]}",
                    'label': i.label,
                    'uri': i.uri,
                    'base': i.base }
            else:
                uri = {
                    'id': f"authority#{request.identifiersLocal}/occupation#{i.label}",
                    'label': i.label,
                    'base': i.base }

            occupations.append(uri)
        doc['occupation'] = occupations
        doc['occupationLabels']  = [i['label'] for i in occupations]

    # fieldOfActivity
    if request.fieldOfActivity:
        fields = list()
        for i in request.fieldOfActivity:
            uri = {
                    'id': f"authority#{request.identifiersLocal}/fieldOfActivity#{i.uri.split('/')[-1]}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base }
            fields.append(uri)
        doc['fieldOfActivity'] = fields

    # identifiesRWO
    if request.identifiesRWO:
        fields = list()
        for i in request.identifiesRWO:
            identifier = i.uri.split("/")[-1]
            uri = {
                    'id': f"authority#{request.identifiersLocal}/identifiesRWO#{identifier}",
                    'uri': i.uri, 
                    'label': i.label, 
                    'base': i.base}
            fields.append(uri)
        doc['identifiesRWO'] = fields

    return doc

In [18]:
doc = MakeDoc(request)

AttributeError: 'SchemaMads' object has no attribute 'hasAffiliation'

In [24]:
element = request.elementList[0]

In [29]:
element.elementLang.label

''

In [10]:
g = Graph()
g.parse('http://id.loc.gov/authorities/names/n79063441')
g.serialize("Márquez.ttl")


<Graph identifier=Nfe14e2834a0b4fe992d4bfbb3a9996fc (<class 'rdflib.graph.Graph'>)>